# Tutorial on running model with Tal's data

In [1]:
#Install from github repository uncomment the following line
#%pip install git+https://github.com/RodrigoZepeda/ABM_Identifiability_python

  Cloning https://github.com/RodrigoZepeda/ABM_Identifiability_python to /private/var/folders/42/2kdf45dd1qz5n7kf9lm8ld9r0000gn/T/pip-req-build-oqqbz080
  Running command git clone --filter=blob:none --quiet https://github.com/RodrigoZepeda/ABM_Identifiability_python /private/var/folders/42/2kdf45dd1qz5n7kf9lm8ld9r0000gn/T/pip-req-build-oqqbz080
  Resolved https://github.com/RodrigoZepeda/ABM_Identifiability_python to commit 588cab6d3dfb948813ce890d8bd2cc785a432bf4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for amro: filename=amro-0.0.2-cp311-cp311-macosx_10_14_x86_64.whl size=99253 sha256=26759f768efcb19aea189d19e4c4ec45cbc7731dd705158f66d50340eb7ca463
  Stored in directory: /private/var/folders/42/2kdf45dd1qz5n7kf9lm8ld9r0000gn/T/pip-ephem-wheel-cache-utjlahxa/wheels/ca/14/9b/8e39dba042c1df6246c3dde140e6cebc04d291ac9b8d4f687c
Successfully built amro
Note: you may need to restar

In [1]:
import pandas as pd
import numpy as np
import amro 

### Data reading

I've already transformed the data so it looks like this:

In [6]:
total_patients_per_ward = pd.read_csv("../data/wards_py.csv")
total_patients_per_ward.head()

,day,ward,count
0,0,1,3.0
1,0,2,8.0
2,0,3,23.0
3,0,4,16.0
4,0,5,3.0


In [7]:
total_patients_per_ward = np.array(total_patients_per_ward)

The same happens with the wards information

In [9]:
ward_matrix = pd.read_csv("../data/patients_py.csv")
ward_matrix.head()

,day,ward,MRN,is_new,weight,next_day
0,0,1,1,1,1.0,1510
1,0,1,2,1,1.0,1511
2,0,1,3,1,1.0,1512
3,0,2,4,1,1.0,1513
4,0,2,5,1,1.0,1514


Column `is_new` is equal to 1 if the patient is its first day (new arrival) or 0 if it was already there the previous day. `next_day` corresponds to the row number where the patient appears in `day + 1`. If they don't appear in `day + 1` they are set to negative. 

In [10]:
initial_patients = ward_matrix[ward_matrix["day"] == 0].shape[0]
ward_matrix = np.array(ward_matrix)

I'm going to test 100 different parameters but one can easily test the same parameters with 100 different initial conditions or the same parameters and same initial conditions

In [11]:
#Create 100 simulations
num_parameters = 100

#Initial colonized values represent the probability that each patient starts up colonized
colonized_day_0 = 0.2*np.ones(shape = (initial_patients, num_parameters))

#Create the parameter matrix
parameters = np.array([
                    np.linspace(0.1, 0.9, num_parameters), #Alpha
                    np.linspace(0.2, 0.5, num_parameters), #Beta
                    np.linspace(0.5, 0.7, num_parameters), #Gamma
                    ]).transpose()

In [12]:
#Run the model 50 times to calculate how much does it take to run on average as well as the standard deviation
#Note that this time is what it took to run not 1 but 100 simulations
%timeit -n1 -r50 amro.simulate_discrete_model_internal_one(colonized_day_0, ward_matrix, total_patients_per_ward, parameters, 1)

17.1 s ± 3.02 s per loop (mean ± std. dev. of 50 runs, 1 loop each)


### Session info

In [13]:
import session_info
session_info.show()